In [1]:
import cv2
import dlib
import imutils
from imutils import face_utils
from scipy.spatial import distance

# Initialize the face detector and predictor
detector = dlib.get_frontal_face_detector()
predictor_path = "shape_predictor_68_face_landmarks.dat"  # Update this path
predictor = dlib.shape_predictor(predictor_path)

def compute(x, y):
    return abs(x - y)  # Return a scalar value (the absolute difference)

def blinked(a, b, c, d, e, f):
    up = compute(b, d) + compute(c, e)  # Both a and b should be scalars
    down = compute(a, f)  # a and f should also be scalars
    ratio = up / (2.0 * down) if down != 0 else 0  # Avoid division by zero

    # Checking if it is blinked
    if ratio > 0.25:
        return 2
    elif 0.21 < ratio <= 0.25:
        return 1
    else:
        return 0

In [2]:
# Initialize video capture
cap = cv2.VideoCapture(0)

sleep = 0
drowsy = 0
active = 0
status = "Active :)"
color = (0, 255, 0)

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    face_frame = frame.copy()  # Initialize face_frame outside the loop

    # Detected face in faces array
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # Extract scalar values for the landmarks used in blink detection
        left_blink = blinked(landmarks[36][1], landmarks[37][1], 
                             landmarks[38][1], landmarks[41][1], 
                             landmarks[40][1], landmarks[39][1])
        right_blink = blinked(landmarks[42][1], landmarks[43][1], 
                              landmarks[44][1], landmarks[47][1], 
                              landmarks[46][1], landmarks[45][1])

        # Now judge what to do for the eye blinks
        if left_blink == 0 or right_blink == 0:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "SLEEPING !!!"
                color = (255, 0, 0)

        elif left_blink == 1 or right_blink == 1:
            sleep = 0
            active = 0
            drowsy += 1
            if drowsy > 6:
                status = "Drowsy !"
                color = (0, 0, 255)

        else:
            drowsy = 0
            sleep = 0
            active += 1
            if active > 6:
                status = "Active :)"
                color = (0, 255, 0)

        cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

        for n in range(0, 68):
            (x, y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    # Display the frames
    cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)

    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 